Pobierz dane dotyczące scoringu kredytowego

*Pracujesz jako analityk danych w globalnej firmie finansowej. Przez lata firma gromadziła podstawowe dane bankowe i wiele informacji związanych z kredytami. Kierownictwo chce zbudować inteligentny system do segregowania ludzi w przedziałach oceny kredytowej, aby zmniejszyć wysiłek ręczny.*

**Zadanie na dziś: przygotować dane do modelu**

In [124]:
#import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

import danych

In [125]:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "train.csv"

# Load the latest version
df = kagglehub.load_dataset(KaggleDatasetAdapter.PANDAS, "parisrohan/credit-score-classification", file_path,
)

print("First 5 records:", df.head())

<ipython-input-125-db52f5663265>:9: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(KaggleDatasetAdapter.PANDAS, "parisrohan/credit-score-classification", file_path,


First 5 records:        ID Customer_ID     Month           Name   Age          SSN Occupation  \
0  0x1602   CUS_0xd40   January  Aaron Maashoh    23  821-00-0265  Scientist   
1  0x1603   CUS_0xd40  February  Aaron Maashoh    23  821-00-0265  Scientist   
2  0x1604   CUS_0xd40     March  Aaron Maashoh  -500  821-00-0265  Scientist   
3  0x1605   CUS_0xd40     April  Aaron Maashoh    23  821-00-0265  Scientist   
4  0x1606   CUS_0xd40       May  Aaron Maashoh    23  821-00-0265  Scientist   

  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  ...  Credit_Mix  \
0      19114.12            1824.843333                  3  ...           _   
1      19114.12                    NaN                  3  ...        Good   
2      19114.12                    NaN                  3  ...        Good   
3      19114.12                    NaN                  3  ...        Good   
4      19114.12            1824.843333                  3  ...        Good   

   Outstanding_Debt Credit_Utiliz

/usr/local/lib/python3.11/dist-packages/kagglehub/pandas_datasets.py:91: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  result = read_function(


**Poznaj dane:**

1.   sprawdź wymiary i typy kolumn
2.   usuń duplikaty
3.   usuń zdbędne kolumny
4.   sprawdz liczbę braków danych w poszczególnych kolumnach






In [126]:
df.info()
df[:10]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good
5,0x1607,CUS_0xd40,June,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,27.262259,22 Years and 6 Months,No,49.574949,62.430172331195294,!@9#%8,340.4792117872438,Good
6,0x1608,CUS_0xd40,July,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,22.537593,22 Years and 7 Months,No,49.574949,178.3440674122349,Low_spent_Small_value_payments,244.5653167062043,Good
7,0x1609,CUS_0xd40,August,NaN,23,#F%$D@*&8,Scientist,19114.12,1824.843333,3,...,Good,809.98,23.933795,NaN,No,49.574949,24.785216509052056,High_spent_Medium_value_payments,358.12416760938714,Standard
8,0x160e,CUS_0x21b1,January,Rick Rothackerj,28_,004-07-5839,_______,34847.84,3037.986667,2,...,Good,605.03,24.464031,26 Years and 7 Months,No,18.816215,104.291825168246,Low_spent_Small_value_payments,470.69062692529184,Standard
9,0x160f,CUS_0x21b1,February,Rick Rothackerj,28,004-07-5839,Teacher,34847.84,3037.986667,2,...,Good,605.03,38.550848,26 Years and 8 Months,No,18.816215,40.39123782853101,High_spent_Large_value_payments,484.5912142650067,Good


In [127]:
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

Usuń kolumny, które nie będą wykorzystywane w analizie ['ID','Customer_ID','SSN','Name']

In [128]:
df = df.drop(['ID','Customer_ID','SSN','Name'],axis=1)


In [129]:
df.info()
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Month                     100000 non-null  object 
 1   Age                       100000 non-null  object 
 2   Occupation                100000 non-null  object 
 3   Annual_Income             100000 non-null  object 
 4   Monthly_Inhand_Salary     84998 non-null   float64
 5   Num_Bank_Accounts         100000 non-null  int64  
 6   Num_Credit_Card           100000 non-null  int64  
 7   Interest_Rate             100000 non-null  int64  
 8   Num_of_Loan               100000 non-null  object 
 9   Type_of_Loan              88592 non-null   object 
 10  Delay_from_due_date       100000 non-null  int64  
 11  Num_of_Delayed_Payment    92998 non-null   object 
 12  Changed_Credit_Limit      100000 non-null  object 
 13  Num_Credit_Inquiries      98035 non-null   fl

usuń niektore z zbędnych znakow ('_','-'), zmień puste komorki na nan

In [130]:
df = df.replace({'_': '', '-': ''}, regex=True)

In [131]:
df.info()
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Month                     100000 non-null  object 
 1   Age                       100000 non-null  object 
 2   Occupation                100000 non-null  object 
 3   Annual_Income             100000 non-null  object 
 4   Monthly_Inhand_Salary     84998 non-null   float64
 5   Num_Bank_Accounts         100000 non-null  int64  
 6   Num_Credit_Card           100000 non-null  int64  
 7   Interest_Rate             100000 non-null  int64  
 8   Num_of_Loan               100000 non-null  object 
 9   Type_of_Loan              88592 non-null   object 
 10  Delay_from_due_date       100000 non-null  int64  
 11  Num_of_Delayed_Payment    92998 non-null   object 
 12  Changed_Credit_Limit      100000 non-null  object 
 13  Num_Credit_Inquiries      98035 non-null   fl

In [132]:
df = df.fillna("nan")

In [133]:
df.isnull().sum()

,0
Month,0
Age,0
Occupation,0
Annual_Income,0
Monthly_Inhand_Salary,0
Num_Bank_Accounts,0
Num_Credit_Card,0
Interest_Rate,0
Num_of_Loan,0
Type_of_Loan,0


In [134]:
df[:10]

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,January,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,HighspentSmallvaluepayments,312.49408867943663,Good
1,February,23,Scientist,19114.12,nan,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,Good,809.98,31.944960,nan,No,49.574949,118.28022162236736,LowspentLargevaluepayments,284.62916249607184,Good
2,March,500,Scientist,19114.12,nan,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,LowspentMediumvaluepayments,331.2098628537912,Good
3,April,23,Scientist,19114.12,nan,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,LowspentSmallvaluepayments,223.45130972736786,Good
4,May,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,HighspentMediumvaluepayments,341.48923103222177,Good
5,June,23,Scientist,19114.12,nan,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,Good,809.98,27.262259,22 Years and 6 Months,No,49.574949,62.430172331195294,!@9#%8,340.4792117872438,Good
6,July,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,Good,809.98,22.537593,22 Years and 7 Months,No,49.574949,178.3440674122349,LowspentSmallvaluepayments,244.5653167062043,Good
7,August,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,Good,809.98,23.933795,nan,No,49.574949,24.785216509052056,HighspentMediumvaluepayments,358.12416760938714,Standard
8,January,28,,34847.84,3037.986667,2,4,6,1,CreditBuilder Loan,...,Good,605.03,24.464031,26 Years and 7 Months,No,18.816215,104.291825168246,LowspentSmallvaluepayments,470.69062692529184,Standard
9,February,28,Teacher,34847.84,3037.986667,2,4,6,1,CreditBuilder Loan,...,Good,605.03,38.550848,26 Years and 8 Months,No,18.816215,40.39123782853101,HighspentLargevaluepayments,484.5912142650067,Good


Zmień nazwy miesiecy na zmienne liczbowe zgodnie z kolejnością miesięcy

In [135]:
month_mapping = {
    "January": 1, "February": 2, "March": 3, "April": 4, "May": 5, "June": 6,
    "July": 7, "August": 8, "September": 9, "October": 10, "November": 11, "December": 12
}


df['Month'] = df['Month'].map(month_mapping)




Zakoduj zmienne tekstowe: 'Occupation', 'Type_of_Loan', 'Payment_Behaviour', 'Credit_Mix', użyj LabelEncodera

In [136]:
le = LabelEncoder()
df['Occupation'] = le.fit_transform(df['Occupation'])
df['Payment_Behaviour'] = le.fit_transform(df['Payment_Behaviour'])
df['Credit_Mix'] = le.fit_transform(df['Credit_Mix'])

In [137]:
df[10:20]

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
10,3,28,14,34847.84,3037.986667,2,1385,6,1,CreditBuilder Loan,...,0,605.03,33.224951,26 Years and 9 Months,No,18.816215,58.51597569589465,1,466.46647639764313,Standard
11,4,28,14,34847.84,nan,2,4,6,1,CreditBuilder Loan,...,2,605.03,39.182656,26 Years and 10 Months,No,18.816215,99.30622796053305,5,465.6762241330048,Good
12,5,28,14,34847.84,3037.986667,2,4,6,1,CreditBuilder Loan,...,2,605.03,34.977895,26 Years and 11 Months,No,18.816215,130.11542024292334,6,444.8670318506144,Good
13,6,28,14,34847.84,3037.986667,2,4,6,1,CreditBuilder Loan,...,2,605.03,33.381010,27 Years and 0 Months,No,18.816215,43.477190144355745,1,481.505261949182,Good
14,7,28,14,34847.84,nan,2,4,6,1,CreditBuilder Loan,...,2,605.03,31.131702,27 Years and 1 Months,NM,18.816215,70.10177420755677,2,464.8806778859809,Good
15,8,28,14,34847.84,3037.986667,2,4,6,1,CreditBuilder Loan,...,2,605.03,32.933856,27 Years and 2 Months,No,18.816215,218.90434353388733,6,356.07810855965045,Good
16,1,34,0,143162.64,12187.22,1,5,8,3,"Auto Loan, Auto Loan, and Not Specified",...,2,1303.01,28.616735,17 Years and 9 Months,No,246.992319,168.413702679309,0,1043.3159778669492,Good
17,2,34,5,143162.64,12187.22,1,5,8,3,"Auto Loan, Auto Loan, and Not Specified",...,2,1303.01,41.702573,17 Years and 10 Months,No,246.992319,232.86038375993544,3,998.8692967863226,Good
18,3,34,0,143162.64,nan,1,5,8,3,"Auto Loan, Auto Loan, and Not Specified",...,2,1303.01,26.519815,17 Years and 11 Months,No,246.992319,10000,3,715.741367403555,Good
19,4,34,5,143162.64,12187.22,1,5,8,3,"Auto Loan, Auto Loan, and Not Specified",...,0,1303.01,39.501648,nan,No,246.992319,825.2162699393922,5,426.5134106068658,Good


Przelicz kolumne Credit_History_Age na liczbę mięsiecy

In [138]:

def convert(text):
  parts = text.split()
  if 'nan' not in parts:
    year = int(parts[0])
    month = int(parts[-2])
    return year*12 + month

df['Credit_History_Age'] = df['Credit_History_Age'].apply(convert)



In [139]:
df.head()

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,1,23,13,19114.12,1824.843333,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,0,809.98,26.822620,265.0,No,49.574949,80.41529543900253,3,312.49408867943663,Good
1,2,23,13,19114.12,nan,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,2,809.98,31.944960,NaN,No,49.574949,118.28022162236736,4,284.62916249607184,Good
2,3,500,13,19114.12,nan,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,2,809.98,28.609352,267.0,No,49.574949,81.699521264648,5,331.2098628537912,Good
3,4,23,13,19114.12,nan,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,2,809.98,31.377862,268.0,No,49.574949,199.4580743910713,6,223.45130972736786,Good
4,5,23,13,19114.12,1824.843333,3,4,3,4,"Auto Loan, CreditBuilder Loan, Personal Loan, ...",...,2,809.98,24.797347,269.0,No,49.574949,41.420153086217326,2,341.48923103222177,Good


In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Month                     100000 non-null  int64  
 1   Age                       100000 non-null  object 
 2   Occupation                100000 non-null  int64  
 3   Annual_Income             100000 non-null  object 
 4   Monthly_Inhand_Salary     100000 non-null  object 
 5   Num_Bank_Accounts         100000 non-null  int64  
 6   Num_Credit_Card           100000 non-null  int64  
 7   Interest_Rate             100000 non-null  int64  
 8   Num_of_Loan               100000 non-null  object 
 9   Type_of_Loan              100000 non-null  object 
 10  Delay_from_due_date       100000 non-null  int64  
 11  Num_of_Delayed_Payment    100000 non-null  object 
 12  Changed_Credit_Limit      100000 non-null  object 
 13  Num_Credit_Inquiries      100000 non-null  ob

In [141]:
# Rozdzielanie wartości po przecinku i usuwanie zbędnych spacji
df['Type_of_Loan'] = df['Type_of_Loan'].str.split(', ')

# Użycie MultiLabelBinarizer do One-Hot Encoding
mlb = MultiLabelBinarizer()
encoded_df = pd.DataFrame(mlb.fit_transform(df['Type_of_Loan']), columns=mlb.classes_)

# Łączenie z oryginalnym DataFrame
df = df.join(encoded_df).drop(columns=['Type_of_Loan'])

print(df.head())

   Month  Age  Occupation Annual_Income Monthly_Inhand_Salary  \
0      1   23          13      19114.12           1824.843333   
1      2   23          13      19114.12                   nan   
2      3  500          13      19114.12                   nan   
3      4   23          13      19114.12                   nan   
4      5   23          13      19114.12           1824.843333   

   Num_Bank_Accounts  Num_Credit_Card  Interest_Rate Num_of_Loan  \
0                  3                4              3           4   
1                  3                4              3           4   
2                  3                4              3           4   
3                  3                4              3           4   
4                  3                4              3           4   

   Delay_from_due_date  ... and Auto Loan and CreditBuilder Loan  \
0                    3  ...             0                      0   
1                   -1  ...             0                      0

Zmien inne zmienne na numeryczne

In [146]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Month                        100000 non-null  int64  
 1   Age                          100000 non-null  object 
 2   Occupation                   100000 non-null  int64  
 3   Annual_Income                100000 non-null  object 
 4   Monthly_Inhand_Salary        100000 non-null  object 
 5   Num_Bank_Accounts            100000 non-null  int64  
 6   Num_Credit_Card              100000 non-null  int64  
 7   Interest_Rate                100000 non-null  int64  
 8   Num_of_Loan                  100000 non-null  object 
 9   Delay_from_due_date          100000 non-null  int64  
 10  Num_of_Delayed_Payment       100000 non-null  object 
 11  Changed_Credit_Limit         100000 non-null  object 
 12  Num_Credit_Inquiries         100000 non-null  object 
 13  

Uzupełnij braki w kolumnie Monthly_Inhand_Salary,  Annual_Income, .. średnią

In [145]:
for column in df.columns:
    mean_value = round(df[column].mean())
    df[column].fillna(mean_value, inplace=True)

<ipython-input-145-64da878168c3>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(mean_value, inplace=True)


TypeError: Could not convert string '...' to numeric

In [ ]:
df[40:50]

Narysuj rozkłady zmiennych 'Annual_Income', 'Monthly_Balance'

usuń wartości odstające (w przypadku wieku wszystkie ujemne i te powyzej 100 lat)

Zwizualizuj korelacje między zmiennymi